<a href="https://colab.research.google.com/github/ihasdapie/roadSignClassification/blob/master/Traffic_Sign_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Traffic Sign Classificatio CNN (GTSRB)
Brian Chen | 2020

In [ ]:
 # grab datasets
!wget -q https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Test_Images.zip
!wget -q https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Training_Images.zip
!wget -q https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Test_GT.zip
#unpack data
!unzip -qq /content/GTSRB_Final_Test_GT.zip
!unzip -qq /content/GTSRB_Final_Test_Images.zip
!unzip -qq /content/GTSRB_Final_Training_Images.zip


In [ ]:
# grab test csv
!wget -q https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Test_GT.zip
!unzip -qq /content/GTSRB_Final_Test_GT.zip
# grab pre-processed dataset from my google drive
!pip install gdown
!gdown --id 1ker1JONZ2MiBDAZAL7fCm618zHHoqXZa --output /content/Processed_GTSRB.zip
!unzip -qq Processed_GTSRB.zip

In [ ]:
# imports
import glob
import numpy as np
from skimage import color, exposure, transform, io
import keras
import pandas
import os
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.optimizers import SGD, adam
from keras import backend as K
import time
import cv2
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split


Using TensorFlow backend.


In [ ]:
# Constants
NUM_CLASSES = 43
IMAGE_SIZE=48  
batch_size=128
nb_epochs=100
lr = 0.0001

In [ ]:
def getLabel(img_path):
    return int(img_path.split('/')[-2])

In [ ]:
# import images & labels
BASE_TRAINING_DIR = "/content/P_GTSRB/Final_Training/Images/"

TRAINING_PATHS = glob.glob(os.path.join(BASE_TRAINING_DIR, '*/*.ppm'))

np.random.shuffle(TRAINING_PATHS)

TRAINING_LABELS = [getLabel(path) for path in TRAINING_PATHS]

images = []
for i in TRAINING_PATHS:
    images.append(cv2.imread(i))

    

In [ ]:
X = np.array(images, dtype='float32')
Y = np.eye(NUM_CLASSES, dtype='uint8')[TRAINING_LABELS]
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
imgGen = ImageDataGenerator(featurewise_center=False,
                            featurewise_std_normalization=False,
                            width_shift_range=0.3,
                            height_shift_range=0.3,
                            zoom_range=0.3,
                            shear_range=0.3,
                            rotation_range=20.,)
imgGen.fit(X_train)

In [ ]:
K.set_image_data_format('channels_last')
def cnn_model():
    model = Sequential()
    model.add(Conv2D(32, (3,3), padding='same', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(256, (3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))


    model.add(Flatten())
    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(NUM_CLASSES, activation='softmax'))
    
    return model


In [ ]:
# with adam, non-tpu
model = cnn_model()


model.compile(loss='categorical_crossentropy',
              optimizer=adam(lr=lr),
              metrics=['accuracy'])



In [ ]:
model.fit_generator(imgGen.flow(X_train, Y_train, batch_size=batch_size), steps_per_epoch=(X_train.shape[0]/batch_size),
                    epochs=(nb_epochs*5), validation_data=(X_val, Y_val),
                    callbacks=[ModelCheckpoint('model4.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)])

Epoch 1/500
246/245 [==============================] - 32s 128ms/step - loss: 3.5161 - accuracy: 0.0880 - val_loss: 3.5745 - val_accuracy: 0.0585

Epoch 00001: val_loss improved from inf to 3.57455, saving model to model4.h5
Epoch 2/500
246/245 [==============================] - 24s 96ms/step - loss: 3.2395 - accuracy: 0.1311 - val_loss: 3.2170 - val_accuracy: 0.1178

Epoch 00002: val_loss improved from 3.57455 to 3.21700, saving model to model4.h5
Epoch 3/500
246/245 [==============================] - 24s 97ms/step - loss: 3.0532 - accuracy: 0.1677 - val_loss: 2.7844 - val_accuracy: 0.1924

Epoch 00003: val_loss improved from 3.21700 to 2.78438, saving model to model4.h5
Epoch 4/500
246/245 [==============================] - 24s 98ms/step - loss: 2.8079 - accuracy: 0.2228 - val_loss: 2.3717 - val_accuracy: 0.3109

Epoch 00004: val_loss improved from 2.78438 to 2.37173, saving model to model4.h5
Epoch 5/500
246/245 [==============================] - 24s 98ms/step - loss: 2.5524 - accur

In [ ]:
!cp /content/drive/My\ Drive/GTSRB/model4.h5 /content/model4.h5

In [ ]:
test_entries = pd.read_csv('/content/GT-final_test.csv', sep=';')
BASE_TEST_DIR = "/content/P_GTSRB/Final_Test/Images/"

X_test = []
y_test = []
i = 0

for file_name, class_id in zip(list(test_entries['Filename']), list(test_entries['ClassId'])):
    img_path = os.path.join(BASE_TEST_DIR, file_name)
    X_test.append(cv2.imread(img_path))
    y_test.append(class_id)

X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
model.load_weights('/content/model4.h5')

In [ ]:
# modelv1, SGD, no generator
predictions = model.predict_classes(X_test)
acc = np.sum(predictions== y_test)/np.size(predictions)
print(acc)

0.8569279493269992


In [ ]:
#modelv1, adam, no generator
predictions = model2.predict_classes(X_test)
acc = np.sum(predictions== y_test)/np.size(predictions)
print(acc)

0.8912905779889153


In [ ]:
#modelv1, adam, imggenerator, 50 epochs
predictions = model2.predict_classes(X_test)
acc = np.sum(predictions== y_test)/np.size(predictions)
print(acc)

0.9297703879651623


In [ ]:
#modelv1, adam, more aggressitve imggenerator
predictions = model.predict_classes(X_test)
acc = np.sum(predictions== y_test)/np.size(predictions)
print(acc)

0.9277117973079968
